# `u8timeseries` Intro

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np

from u8timeseries import TimeSeries

## `TimeSeries` Creation
### From a `pd.Series`:

In [ ]:
pdseries = pd.Series(np.sin(range(30)), index=pd.date_range('20130101', '20130130'))
series = TimeSeries(pdseries)

### From a `pd.DataFrame`:

In [ ]:
df = pd.DataFrame({'Date': pd.date_range('20130101', '20130130'), 'Value': np.sin(range(30))})

df.head()

In [ ]:
series = TimeSeries.from_dataframe(df, time_col='Date', value_col='Value')

### From times and values directly:

In [ ]:
series = TimeSeries.from_times_and_values(pd.date_range('20130101', '20130130'), np.sin(range(30)))

### Using pre-defined `TimeSeries` generators:

In [ ]:
from u8timeseries.utils import timeseries_generation as tg

tg.sine_timeseries(length=70).plot()
tg.us_holiday_timeseries(length=70).plot()
(tg.linear_timeseries(length=70) / 100.).plot()

## More on `TimeSeries`
### Plotting
wrapper around `pd.Series.plot()`

In [ ]:
series.plot()

### Confidence Intervals

In [ ]:
pdseries_low = pd.Series(np.sin(range(30)) - 0.3, index=pd.date_range('20130101', '20130130'))
pdseries_up = pd.Series(np.sin(range(30)) + 0.3, index=pd.date_range('20130101', '20130130'))

series_conf = TimeSeries(pdseries, pdseries_low, pdseries_up)

series_conf.plot()

### Drop, Slice, split

In [ ]:
# Drop end:
series.plot()
beginning = series.drop_after(pd.Timestamp('20130115'))
beginning.plot(marker='o', lw=3, alpha=0.5)

In [ ]:
# Slice
subseries = series.slice(pd.Timestamp('20130107'), pd.Timestamp('20130114'))
series.plot()
subseries.plot(marker='o', lw=4, alpha=0.5)

In [ ]:
# Split
s1, s2 = series.split_after(pd.Timestamp('20130110'))
s1.plot()
s2.plot()

### Intersect (in time)

In [ ]:
series1 = TimeSeries(pd.Series(np.sin(range(32)), index=pd.date_range('20130115', '20130215')))
series2 = TimeSeries(pd.Series(np.cos(range(28)), index=pd.date_range('20130201', '20130228')))

series1.plot()
series2.plot()

slice_series = series1.slice_intersect(series2)
slice_series.plot(lw=5, alpha=0.3)

`TimeSeries.intersect()` returns a slice of the time index, which can then be used for indexing:

In [ ]:
series1.plot()
series2.plot()

# no idea the intent here
series2[series1.intersect(series2)].plot(lw=5, alpha=0.3)

# Auto-regressive Models

In [ ]:
df = pd.read_csv('AirPassengers.csv')
series = TimeSeries.from_dataframe(df, 'Month', '#Passengers')
series.plot()

In [ ]:
from u8timeseries import Theta, ExponentialSmoothing

train, val = series.split_after(pd.Timestamp('19580101'))

model_theta = Theta(theta=4)
model_ets = ExponentialSmoothing()

model_theta.fit(train)
model_ets.fit(train)

pred_val_theta = model_theta.predict(len(val))
pred_val_ets = model_ets.predict(len(val))

In [ ]:
from u8timeseries import KthValueAgoBaseline

model_kth = KthValueAgoBaseline(k=1)
model_kth.fit(train)
pred = model_kth.predict(len(val))

series.plot()
pred.plot()

In [ ]:
from u8timeseries.metrics import mape
import matplotlib.pyplot as plt

series.plot(label='actual')
pred_val_theta.plot(label='Theta')
pred_val_ets.plot(label='ETS')
plt.legend()

print('MAPE error Theta = {}'.format(mape(series.slice_intersect(pred_val_theta), pred_val_theta)))
print('MAPE error ETS = {}'.format(mape(series.slice_intersect(pred_val_ets), pred_val_ets)))

### Backtesting

In [ ]:
from u8timeseries.backtesting import simulate_forecast_ar

historical_fcast = simulate_forecast_ar(series, model_ets, pd.Timestamp('19570601'), fcast_horizon_n=6, verbose=True)

In [ ]:
series.plot(lw=2, alpha=0.5)
historical_fcast.plot()

### Metrics

In [ ]:
from u8timeseries.metrics import mape, mase

slice_series = series.intersect(historical_fcast)
mape(series[slice_series], historical_fcast[slice_series])